<a href="https://colab.research.google.com/github/shiehn/elixirs/blob/main/elixir_audio_two_midi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tempfile
import os

# Check if ffmpeg is installed, if not then install it
ffmpeg_installed = !command -v ffmpeg
if not ffmpeg_installed:
    !apt-get install ffmpeg

#Check if TensorFlow is installed, if not then install il
try:
  import tensorflow as tf
except ImportError:
  !pip install tensorflow
  import tensorflow as tf

#Check if BASIC_PITCH is installed, if not then install it
try:
  from basic_pitch.inference import predict
  from basic_pitch import ICASSP_2022_MODEL_PATH
except ImportError:
  !pip install basic-pitch

from basic_pitch.inference import predict
from basic_pitch import ICASSP_2022_MODEL_PATH

#LOAD THE BASIC_PITCH MODEL
basic_pitch_model = tf.saved_model.load(str(ICASSP_2022_MODEL_PATH))
if basic_pitch_model is None:
  print("ERROR: The basic_pitch model could not be loaded.")
  exit()

#Check if dawnet-client is installed, if not then install it
try:
    import dawnet_client
except ImportError:
    !pip install dawnet-client
    import dawnet_client

import dawnet_client.core as dawnet
from dawnet_client import DAWNetFilePath


placeholder_txt = "Enter the token generated by the DAWNet plugin"
DAWNET_TOKEN = "DAWNET_TOKEN" #@param {type:"string"}
dawnet_token = DAWNET_TOKEN

if dawnet_token is None or dawnet_token == "" or dawnet_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

async def dawnet_func(input_file: DAWNetFilePath):
    try:
        model_output, midi_data, note_events = predict(input_file,basic_pitch_model)

        midi_data.write("midi_data.mid")

        if os.path.exists("midi_data.mid"):
            await dawnet.output().add_log("Midi file created")
        else:
            await dawnet.output().add_log("File does not exist")
            await dawnet.output().add_error("MIDI not created")
            return False

        await dawnet.output().add_file("midi_data.mid")
        await dawnet.output().add_message("AUDIO TO MIDI - COMPLETE")
        await dawnet.output().send()

        return True
    except Exception as e:
        await dawnet.output().add_error(f"Error in arbitrary_method: {e}")

        return False


dawnet.set_token(token=dawnet_token)
dawnet.set_name("DAWNet AUDIO_2_MIDI")
dawnet.set_description("This is wrapper on Spotifies BASIC-PITCH. Input an audio file and get back a midi file.")
dawnet.register_method(dawnet_func)

dawnet.connect_to_server()